### Setup
Configuration, Logger and Downloader

In [ ]:
import os
import logging
import time

import pandas as pd
from core.config import Config
from data.cleaning import read_all_csv
from data.download import LSEGDataDownloader
os.environ["RD_LIB_CONFIG_PATH"] = "/Configuration"

config = Config()
logging.basicConfig(
        filename=config.log_file,
        encoding="utf-8",
        level=config.log_level,
        format='%(asctime)s %(levelname)-8s %(message)s',
        datefmt = '%Y-%m-%d %H:%M:%S'
    )
logger = logging.getLogger()

### Downloading time series Data
Beginning from 8766.T

In [ ]:
with (LSEGDataDownloader(config) as downloader):
    logger.info("Downloading historic frames from LSEG database beginning from 8766.T")
    historic: dict[str, pd.DataFrame] = {}
    for chunk in config.companies_historic_chunks:
        historic_result: dict[str, pd.DataFrame] = (
            downloader.download_all_historic_chunks(chunk)
        )
        historic.update(historic_result)

### Downloading static Data

In [ ]:
with LSEGDataDownloader(config) as downloader:
    for companies in config.companies_static_chunks[34:]:
        statdict: dict[str, pd.DataFrame] = downloader.download_static_from(companies, config.static_features)
        for name, frame in statdict.items():
            frame.to_csv(config.static_dir / f"company-{name}.csv")
        time.sleep(60)

### Building holy Frame

In [ ]:
dictionary = read_all_csv(config.historic_dir)

In [ ]:
holy_frame: pd.DataFrame = pd.concat(dictionary.values())

In [ ]:
dup_names = holy_frame.columns[holy_frame.columns.duplicated()].unique().tolist()
print(dup_names)